In [ ]:
from huggingface_hub import notebook_login
import pandas as pd
import librosa
import datasets
import numpy as np
from datasets import Dataset
from datasets import Audio
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from huggingface_hub import login
api_token = ""
login(token = api_token, add_to_git_credential=True)

In [ ]:
df = pd.read_csv("")

def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)
    signal_array = np.array(signal)
    return {"path": file_path, "array": signal_array, "sampling_rate": sr}

def load_dataset(dataframe):
    dataset = Dataset.from_pandas(dataframe)
    dataset = dataset.map(
        lambda example: {
            "audio": load_audio(example["audio"]),
            "sentence": example["sentence"]
        },
    )
    return dataset

In [ ]:
df_valid = pd.read_csv("")

def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)
    signal_array = np.array(signal)
    return {"path": file_path, "array": signal_array, "sampling_rate": sr}

def load_dataset(dataframe):
    dataset = Dataset.from_pandas(dataframe)
    dataset = dataset.map(
        lambda example: {
            "audio": load_audio(example["audio"]),
            "sentence": example["sentence"]
        },
    )
    return dataset

In [ ]:

data_dict = datasets.DatasetDict({
    "train": load_dataset(df.iloc[:int(len(df)*1)]),
    "test": load_dataset(df_valid.iloc[:int(len(df_valid)*1)])
})


In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3", language="romanian", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language="romanian", task="transcribe")

In [ ]:
data_dict = data_dict.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
mds = 30.0
mil = int(mds * 16000)

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute input length
    batch["input_length"] = len(batch["audio"])

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"], max_length=mil).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"], truncation=True,max_length=448).input_ids

    # compute labels length
    batch["labels_length"] = len(tokenizer(batch["sentence"], add_special_tokens=False).input_ids)
    return batch


In [ ]:
data_dict = data_dict.map(prepare_dataset,remove_columns=data_dict.column_names["train"],num_proc=1)


In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "romanian", task = "transcribe")
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="",  
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,  
    learning_rate=2e-5,
    lr_scheduler_type="linear",  
    warmup_steps=3000,
    max_steps=20000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    save_total_limit=3, 
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()